In [43]:
import numpy as np

def lbp_1d(signal, komsuluk):
    binmap = []

    l = len(signal)

    for i in range(l):
        kon = np.zeros(2 * komsuluk)
        kon[:min(komsuluk, i + 1)] = signal[max(0, i - komsuluk + 1):i + 1]

        dat = np.zeros(2 * komsuluk)
        start_index = max(0, i - komsuluk + 1)
        end_index = min(i + komsuluk + 1, l)  # Fixed the end_index calculation
        dat[:end_index - start_index] = signal[start_index:end_index]

        binmap.append((kon > dat).astype(int))

    binmap = [int(''.join(map(str, binmap[i][::-1])), 2) for i in range(len(binmap))]  # Reverse the order

    return binmap

# Example usage:
signal_example = np.random.randn(100)
komsuluk_example = 4
result_binmap = lbp_1d(signal_example, komsuluk_example)

print(signal_example)
print("Binary Map:")
print(result_binmap)


[ 0.34119835 -1.06211718 -1.48387907 -0.59026846  0.19489659  0.64636504
 -0.25015569  0.11398083  0.42841865 -0.29300531 -0.61180795 -0.15955416
 -1.20707574  0.63451149 -0.59701931  1.19608933 -0.25744779  0.17206486
 -1.12125451 -0.37187384  0.07164407  2.66712465  1.17039283 -0.45275612
  0.52246342  0.92961672 -0.81498648  0.90040386  0.42882892 -1.38118272
  0.54906281  0.08908265  0.03264905 -0.177779    0.52809438  1.51963625
 -0.79901864 -2.36707502  0.50737232  0.17852863  1.72300647 -0.43386617
  1.71782669  0.54135538  0.10563704  1.3426537   0.80793468  0.91919896
  0.01307349 -1.18337286  0.18067111 -1.60239973  0.46186919 -0.62584466
  1.21972586  0.07482265 -1.09900757  0.19035591  1.91600073  0.4183303
  0.47947343 -0.52271992 -0.91604598 -0.26754477  0.9722494   0.20501059
  0.17418548 -1.3567568   0.57742687  0.92129073  0.02407163 -0.15490437
 -1.00531815 -0.2707591   0.3826001  -0.65993181 -0.87703393  0.60314232
  1.08869305 -0.83620552  1.62817657  0.52194395  0.

In [39]:
import numpy as np
from collections import Counter

with open('spam/D1.txt', 'r') as file:
    A = file.readlines()

with open('spam/D2.txt', 'r') as file:
    B = file.readlines()

features = []
komsuluk=4
for i in range(len(A)):
    dosya = A[i].strip()
    bytes = [ord(char) for char in dosya]

    if len(bytes) > 10:
        veri = bytes

        b = lbp_1d(veri,komsuluk )
        count = dict(Counter(b))

        if B[i].strip() == 'ham':
            label = 1
        else:
            label = 0

        # Ensure all 'count' arrays have the same length
        max_length = 256
        padded_count = np.zeros(max_length)
        padded_count[list(count.keys())] = list(count.values())

        features.append(np.concatenate((padded_count, [label])))

# Convert the features list to a NumPy array
features = np.array(features)

# Print or use the 'features' array as needed
print(features.shape)


(1233, 257)


In [44]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# Giriş öznitelikleri ve çıkış sütununu ayırma
X = features[:, :features.shape[1]-1]  
y = features[:, features.shape[1]-1]   # Son sütun çıkış


# Veriyi eğitim ve test setlerine bölelim
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# KNN sınıflandırıcı
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
print("KNN Başarı:", knn_accuracy)

knn_conf_matrix = confusion_matrix(y_test, knn_predictions)
print("KNN Karışıklık Matrisi:")
print(knn_conf_matrix)

# Naive Bayes sınıflandırıcı
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
nb_predictions = nb_classifier.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_predictions)
print("Naive Bayes Başarı", nb_accuracy)

nb_conf_matrix = confusion_matrix(y_test, nb_predictions)
print("Naive Bayes Karışıklık Matrisi:")
print(nb_conf_matrix)

KNN Başarı: 0.8918918918918919
KNN Karışıklık Matrisi:
[[ 67  17]
 [ 23 263]]
Naive Bayes Başarı 0.8756756756756757
Naive Bayes Karışıklık Matrisi:
[[ 70  14]
 [ 32 254]]
